In [5]:


import requests
import json
import os

REGION = "us"
LOCALE = "en_US"
DESKTOP_PATH = os.path.join(os.path.expanduser("~"), "OneDrive", "Desktop")
CLIENT_ID = "a98d49dee4924e12bea4b0df7821862b"
CLIENT_SECRET = "5J8AS1ichT3A8GPxAgU1Bmrus5wA4WK4"

# --- Obtener token ---
def get_token():
    r = requests.post(
        f"https://{REGION}.battle.net/oauth/token",
        auth=(CLIENT_ID, CLIENT_SECRET),
        data={"grant_type": "client_credentials"}
    )
    r.raise_for_status()
    return r.json()["access_token"]

# --- Buscar todos los reinos ---
def list_realms(token):
    realms = []
    page = 1

    while True:
        r = requests.get(
            f"https://{REGION}.api.blizzard.com/data/wow/search/realm",
            headers={"Authorization": f"Bearer {token}"},
            params={
                "namespace": f"dynamic-{REGION}",
                "locale": LOCALE,
                "_page": page,
                "_pageSize": 100,
                "orderby": "id"
            }
        )
        r.raise_for_status()
        data = r.json()

        if "results" not in data or len(data["results"]) == 0:
            break

        for item in data["results"]:
            realms.append({
                "id": item["data"]["id"],
                "name": item["data"]["name"]["en_US"],
                "slug": item["data"]["slug"]
            })

        page += 1

    return realms

# --- Main ---
def main():
    token = get_token()
    realms = list_realms(token)

    with open(os.path.join(DESKTOP_PATH, "wow_realms.json"), "w", encoding="utf-8") as f:
        json.dump(realms, f, indent=2)
    
    print(f"💾 {len(realms)} reinos guardados en wow_realms.json")

main()



💾 344 reinos guardados en wow_realms.json
